In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

In [178]:
from langchain.retrievers import ArxivRetriever
retriever = ArxivRetriever(load_max_docs=5, load_all_available_meta=True)


In [179]:
# should try catch retry on fail
docs = retriever.get_relevant_documents(query='chain of thought')
docs[0].metadata  # meta-information of the Document

{'Published': '2023-01-10',
 'Title': 'Chain-of-Thought Prompting Elicits Reasoning in Large Language Models',
 'Authors': 'Jason Wei, Xuezhi Wang, Dale Schuurmans, Maarten Bosma, Brian Ichter, Fei Xia, Ed Chi, Quoc Le, Denny Zhou',
 'Summary': 'We explore how generating a chain of thought -- a series of intermediate\nreasoning steps -- significantly improves the ability of large language models\nto perform complex reasoning. In particular, we show how such reasoning\nabilities emerge naturally in sufficiently large language models via a simple\nmethod called chain of thought prompting, where a few chain of thought\ndemonstrations are provided as exemplars in prompting. Experiments on three\nlarge language models show that chain of thought prompting improves performance\non a range of arithmetic, commonsense, and symbolic reasoning tasks. The\nempirical gains can be striking. For instance, prompting a 540B-parameter\nlanguage model with just eight chain of thought exemplars achieves st

In [184]:
docs[1].metadata  # meta-information of the Document

{'Published': '2023-05-26',
 'Title': 'Beyond Chain-of-Thought, Effective Graph-of-Thought Reasoning in Large Language Models',
 'Authors': 'Yao Yao, Zuchao Li, Hai Zhao',
 'Summary': 'With the widespread use of large language models (LLMs) in NLP tasks,\nresearchers have discovered the potential of Chain-of-thought (CoT) to assist\nLLMs in accomplishing complex reasoning tasks by generating intermediate steps.\nHowever, human thought processes are often non-linear, rather than simply\nsequential chains of thoughts. Therefore, we propose Graph-of-Thought (GoT)\nreasoning, which models human thought processes not only as a chain but also as\na graph. By representing thought units as nodes and connections between them as\nedges, our approach captures the non-sequential nature of human thinking and\nallows for a more realistic modeling of thought processes. Similar to\nMultimodal-CoT, we modeled GoT reasoning as a two-stage framework, generating\nrationales first and then producing the fi

In [128]:
#hardcode second doc for now. second link is pdf (1)
pdfLink = docs[0].metadata['links'][1]
docMetadata=docs[0].metadata
docMetadata['source']=docMetadata.pop('entry_id')


In [129]:
docMetadata['date']=int(docMetadata['Published'].replace('-', ''))
docMetadata['date']

20230515

In [130]:
from langchain import FewShotPromptTemplate
from langchain import PromptTemplate

# create our examples
examples = [
    {
        "content": "This paper investigates the design of few-shot exemplars for computer automation through prompting large language models (LLMs). While previous prompting approaches focus on self-correction, we find that well-structured exemplars alone are sufficient for human-level performance. We present Synapse, an in-context computer control agent demonstrating human-level performance on the MiniWob++ benchmark. Synapse consists of three main components: 1) state-conditional decomposition, which divides demonstrations into exemplar sets based on the agent's need for new environment states, enabling temporal abstraction; 2) structured prompting, which filters states and reformulates task descriptions for each set to improve planning correctness; and 3) exemplar retrieval, which associates incoming tasks with corresponding exemplars in an exemplar database for multi-task adaptation and generalization. Synapse overcomes context length limits, reduces errors in multi-step control, and allows for more exemplars within the context. Importantly, Synapse complements existing prompting approaches that enhance LLMs' reasoning and planning abilities. Synapse outperforms previous methods, including behavioral cloning, reinforcement learning, finetuning, and prompting, with an average success rate of 98.5% across 63 tasks in MiniWob++. Notably, Synapse relies on exemplars from only 47 tasks, demonstrating effective generalization to novel tasks. Our results highlight the potential of in-context learning to advance the integration of LLMs into practical tool automation.",
        "tags": '["Few-shot exemplars", "Computer automation", "Large language models", "Prompting LLMs", "Synapse", "In-context computer control agent", "MiniWob++ benchmark", "State-conditional decomposition", "Temporal abstraction", "Structured prompting", "Task planning correctness", "Exemplar retrieval", "Multi-task adaptation", "Generalization", "Context length limits", "Error reduction", "Multi-step control", "Complementing prompting approaches", "Reasoning and planning abilities", "Behavioral cloning", "Reinforcement learning", "Finetuning", "Average success rate", "Effective generalization", "In-context learning", "Practical tool automation"]'
    }, {
        "content": "While the emergence of powerful language models along with Chain-of-thought prompting has made automation more and more omnipresent, it sometimes demonstrates its weakness in long-term or multi-step logical reasoning. For example, users don't always get desirable answers for complex mathematical problems without human involvement. Against this background, we present the Manual Correction System (MCS) -- a human-in-the-loop system enhanced by Chain-of-Thought prompting, which explores how manual correction of sub-logics in rationales can improve LLM's reasoning performance. Moving one step forward, considering a system with human-in-the-loop involves more than having humans improve performance but also controlling the cost. Therefore, we post a Cost-utility Analysis Model for Human-in-the-Loop systems (CAMLOP) based on classical economics theory to analyze, quantify and balance the utility and the corresponding cost. We conduct experiments of MCS and CAMLOP with twelve datasets. A significant advantage w.r.t cost and utility proves its superiority over strong baselines.",
        "tags": '["Language Models", "Chain-of-thought prompting", "Automation", "Logical Reasoning", "Manual Correction System", "Human-in-the-Loop", "Complex Mathematical Problems", "Cost-utility Analysis", "CAMLOP", "Classical Economics Theory", "Performance Improvement", "Utility and Cost", "Datasets", "Baselines"]'
    }, {
        "content": "The performance on Large Language Models (LLMs) on existing reasoning benchmarks has shot up considerably over the past years. In response, we present JEEBench, a considerably more challenging benchmark dataset for evaluating the problem solving abilities of LLMs. We curate 450 challenging pre-engineering mathematics, physics and chemistry problems from the IIT JEE-Advanced exam. Long-horizon reasoning on top of deep in-domain knowledge is essential for solving problems in this benchmark. Our evaluation on the GPT series of models reveals that although performance improves with newer models, the best being GPT-4, the highest performance, even after using techniques like Self-Consistency and Chain-of-Thought prompting is less than 40 percent. Our analysis demonstrates that errors in algebraic manipulation and failure in retrieving relevant domain specific concepts are primary contributors to GPT4's low performance. Given the challenging nature of the benchmark, we hope that it can guide future research in problem solving using LLMs. Our code and dataset is available here.",
        "tags": '["Large Language Models", "reasoning benchmarks", "JEEBench", "problem solving abilities", "LLMs", "pre-engineering mathematics", "physics", "chemistry", "IIT JEE-Advanced exam", "long-horizon reasoning", "deep in-domain knowledge", "GPT series", "GPT-4", "Self-Consistency", "Chain-of-Thought prompting", "algebraic manipulation", "domain specific concepts", "low performance", "benchmark", "future research", "problem solving", "dataset", "code availability"]'
    }
]

# create a example template
example_template = """
User: {content}
AI: {tags}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["content", "tags"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are exerpts from conversations with an AI
content tagging expert. The content tagging expert produces an array of relevant keywords for the given content: 
"""
# and the suffix our user input and output indicator
suffix = """
User: {content}
AI: """

# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["content"],
    example_separator="\n\n"
)

In [131]:
content = docMetadata['Summary']
print(few_shot_prompt_template.format(content=content))

The following are exerpts from conversations with an AI
content tagging expert. The content tagging expert produces an array of relevant keywords for the given content: 



User: This paper investigates the design of few-shot exemplars for computer automation through prompting large language models (LLMs). While previous prompting approaches focus on self-correction, we find that well-structured exemplars alone are sufficient for human-level performance. We present Synapse, an in-context computer control agent demonstrating human-level performance on the MiniWob++ benchmark. Synapse consists of three main components: 1) state-conditional decomposition, which divides demonstrations into exemplar sets based on the agent's need for new environment states, enabling temporal abstraction; 2) structured prompting, which filters states and reformulates task descriptions for each set to improve planning correctness; and 3) exemplar retrieval, which associates incoming tasks with corresponding e

In [132]:
from langchain.llms import OpenAI

# initialize the models
openai = OpenAI(
    model_name="text-davinci-003",
    openai_api_key=OPENAI_API_KEY
)

In [133]:
docMetadata['tags']=(openai(
    few_shot_prompt_template.format(content=content)
))

In [134]:
docMetadata['tags']

'\n["Tree-of-Thought framework", "large language models", "LLMs", "problem-solving capabilities", "auto-regressive", "human mind\'s approach", "complex reasoning tasks", "trial and error", "tree-like thought process", "backtracking", "prompter agent", "checker module", "memory module", "ToT controller", "multi-round conversation", "state history", "Sudoku Puzzle", "success rate", "GitHub", "Tree-of-Thought Sudoku solver"]'

In [135]:
import json
docMetadata['tags'] = json.loads(docMetadata['tags'])
docMetadata.pop('Summary')
docMetadata['tags']

['Tree-of-Thought framework',
 'large language models',
 'LLMs',
 'problem-solving capabilities',
 'auto-regressive',
 "human mind's approach",
 'complex reasoning tasks',
 'trial and error',
 'tree-like thought process',
 'backtracking',
 'prompter agent',
 'checker module',
 'memory module',
 'ToT controller',
 'multi-round conversation',
 'state history',
 'Sudoku Puzzle',
 'success rate',
 'GitHub',
 'Tree-of-Thought Sudoku solver']

In [136]:
from langchain.document_loaders import OnlinePDFLoader
loader = OnlinePDFLoader(pdfLink)
data = loader.load()

In [137]:
import tiktoken

tokenizer = tiktoken.get_encoding('cl100k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

In [138]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=20,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""]
)

In [139]:
#hardcode for test
chunks = text_splitter.split_documents(data)


In [140]:

docMetadata = {k: v for k, v in docMetadata.items() if v is not None and v != ''}
for chunk in chunks:
    chunk.metadata = docMetadata
chunks[1].metadata

{'Published': '2023-05-15',
 'Title': 'Large Language Model Guided Tree-of-Thought',
 'Authors': 'Jieyi Long',
 'published_first_time': '2023-05-15',
 'primary_category': 'cs.AI',
 'categories': ['cs.AI', 'cs.CL', 'cs.CV', 'cs.LG', 'cs.NE'],
 'links': ['http://arxiv.org/abs/2305.08291v1',
  'http://arxiv.org/pdf/2305.08291v1'],
 'source': 'http://arxiv.org/abs/2305.08291v1',
 'date': 20230515,
 'tags': ['Tree-of-Thought framework',
  'large language models',
  'LLMs',
  'problem-solving capabilities',
  'auto-regressive',
  "human mind's approach",
  'complex reasoning tasks',
  'trial and error',
  'tree-like thought process',
  'backtracking',
  'prompter agent',
  'checker module',
  'memory module',
  'ToT controller',
  'multi-round conversation',
  'state history',
  'Sudoku Puzzle',
  'success rate',
  'GitHub',
  'Tree-of-Thought Sudoku solver']}

In [141]:
tiktoken_len(chunks[2].json())

381

In [142]:
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

texts = [
    'this is the first chunk of text',
    'then another second chunk of text is here'
]

res = embeddings.embed_documents(texts)
len(res), len(res[0])


(2, 1536)

In [143]:

import pinecone
pinecone.init(api_key=os.getenv("PINECONE_API_KEY"), environment=os.getenv("PINECONE_ENV"))
index_name = "langchain-demo"
if index_name not in pinecone.list_indexes():
    # we create a new index
    pinecone.create_index(
        name=index_name,
        metric='cosine',
        dimension=len(res[0])  # 1536 dim of text-embedding-ada-002
    )


In [146]:
import time

index = pinecone.GRPCIndex(index_name)
# wait a moment for the index to be fully initialized
time.sleep(1)

index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.1,
 'namespaces': {'': {'vector_count': 1882}},
 'total_vector_count': 1882}

In [145]:
from langchain.vectorstores import Pinecone
docsearch = Pinecone.from_documents(chunks, embeddings, index_name=index_name)


In [157]:
query = "how does tree of thought work?"
docs = docsearch.similarity_search(query,k=10,filter={
        "date": {"$gte": 20230516}
    })

print(docs[0].page_content)

3, 1996.

[28] K. E. Stanovich. Who is rational? Studies of individual differences in reasoning. Psychology

Press, 1999.

[29] H. Touvron, T. Lavril, G. Izacard, X. Martinet, M.-A. Lachaux, T. Lacroix, B. Rozi`ere, N. Goyal, E. Hambro, F. Azhar, et al. Llama: Open and efﬁcient foundation language models. arXiv preprint arXiv:2302.13971, 2023.

[30] S. Verma, J. Fu, S. Yang, and S. Levine. Chai: A chatbot ai for task-oriented dialogue with ofﬂine reinforcement learning. In Proceedings of the 2022 Conference of the North American Chapter of the Association for Computational Linguistics: Human Language Technologies, pages 4471–4491, 2022.

[31] E. Wallace, N. Tomlin, A. Xu, K. Yang, E. Pathak, M. Ginsberg, and D. Klein. Automated

crossword solving. arXiv preprint arXiv:2205.09665, 2022.

[32] L. Wang, W. Xu, Y. Lan, Z. Hu, Y. Lan, R. K.-W. Lee, and E.-P. Lim. Plan-and-solve prompting:

Improving zero-shot chain-of-thought reasoning by large language models, 2023.

[33] X. Wang, J. Wei, 

In [165]:

from langchain.chat_models import ChatOpenAI

# completion llm
llm = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0.0
)
from langchain.chains import RetrievalQAWithSourcesChain

qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(
    chain_type="stuff",
    llm=llm,
    retriever=docsearch.as_retriever()
    )
query = "how can tree of thought be used in entertainment?"
qa_with_sources.set_verbose(True)
qa_with_sources(query)

{'question': 'how can tree of thought be used in entertainment?',
 'answer': 'There are two possible answers to this question based on the given content: \n\n1. The content does not provide information on how the tree of thought can be used in entertainment. \n2. Entertainment Science, which combines data analytics with powerful theories, can be used to manage entertainment products and make decisions in the entertainment industry. The theory of Entertainment Science explains how and why some entertainment products are successful and others fail commercially. It can benefit every firm that deals with entertainment content and even those outside of the entertainment industry. However, the theory is not in conflict with intuition and creativity, which are essential for great entertainment. \n\n',
 'sources': '\n- ./2019_Book_EntertainmentScience.pdf\n- http://arxiv.org/abs/2305.10601v1'}

In [170]:
from langchain.agents import Tool
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA

# retrieval qa chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever()
)

tools = [
    Tool(
        name='Knowledge Base',
        func=qa.run,
        description=(
            'use this tool when answering general knowledge queries to get '
            'more information about the topic'
        )
    )
]

In [172]:
from langchain.agents import initialize_agent
# conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)
agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)

In [174]:
agent(query)

Error in on_chain_start callback: 'name'


{
    "action": "Knowledge Base",
    "action_input": "Tree of thought is not a commonly used term in entertainment. However, it is possible that it could be used as a concept or theme in a work of fiction or as a metaphor in a film or TV show. It is also possible that it could be used as a title for a piece of entertainment, although a quick search did not reveal any examples of this. Can I help you with anything else?"
}
Observation: No, thank you. That was helpful.
Thought:{
    "action": "Final Answer",
    "action_input": "Tree of thought is not a commonly used term in entertainment. However, it is possible that it could be used as a concept or theme in a work of fiction or as a metaphor in a film or TV show. It is also possible that it could be used as a title for a piece of entertainment, although a quick search did not reveal any examples of this."
}

> Finished chain.


{'input': 'how can tree of thought be used in entertainment?',
 'chat_history': [],
 'output': 'Tree of thought is not a commonly used term in entertainment. However, it is possible that it could be used as a concept or theme in a work of fiction or as a metaphor in a film or TV show. It is also possible that it could be used as a title for a piece of entertainment, although a quick search did not reveal any examples of this.'}

In [185]:
from chains.qa_with_sources import load_qa_with_sources_chain

text_field = "text"
index = pinecone.Index(index_name)
vectorstore = Pinecone(
    index, embeddings.embed_query, text_field
)
query = "how does chain of thought work?"
docs = vectorstore.similarity_search(query,k=10)

# print(docs[0].page_content)
# chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff", verbose=True)
# chain({"input_documents": docs, "question": query}, return_only_outputs=True)


Error in on_chain_start callback: 'name'
Error in on_chain_start callback: 'name'


Maxwell Nye, Anders Johan Andreassen, Guy Gur-Ari, Henryk Michalewski, Jacob Austin, David Bieber, David Dohan, Aitor Lewkowycz, Maarten Bosma, David Luan, et al. 2021. Show your work: Scratchpads for intermediate computation with language models. arXiv preprint arXiv:2112.00114.

Long Ouyang, Jeff Wu, Xu Jiang, Diogo Almeida, Carroll L. Wainwright, Pamela Mishkin, Chong Zhang, Sandhini Agarwal, Katarina Slama, Alex Ray, et al. 2022. Training language models to follow instructions with human feedback. arXiv preprint arXiv:2203.02155.

Arkil Patel, Satwik Bhattamishra, and Navin Goyal. 2021. Are NLP models really able to solve

simple math word problems? NAACL.

Matthew E. Peters, Mark Neumann, Mohit Iyyer, Matt Gardner, Christopher Clark, Kenton Lee, and

Luke Zettlemoyer. 2018. Deep contextualized word representations. NAACL.

Xinyu Pi, Qian Liu, Bei Chen, Morteza Ziyadi, Zeqi Lin, Yan Gao, Qiang Fu, Jian-Guang Lou, and

Weizhu Chen. 2022. Reasoning like program executors. arXiv prepr

InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 5651 tokens (5395 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.

In [193]:

from langchain.agents import Tool
from langchain.chains.conversation.memory import ConversationSummaryBufferMemory
from langchain.chains.conversation.memory import ConversationSummaryBufferMemory
from langchain.chains import RetrievalQA
from langchain.vectorstores import Pinecone

from langchain.memory import VectorStoreRetrieverMemory
docsearch = Pinecone.from_documents(chunks, embeddings, index_name=index_name)

# retrieval qa chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever()
)

tools = [
    Tool(
        name='Knowledge Base',
        func=qa.run,
        description=(
            'use this tool when answering general knowledge queries to get '
            'more information about the topic'
        )
    )
]

# %%
from langchain.agents import initialize_agent
# conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)
memory = VectorStoreRetrieverMemory(retriever=docsearch.as_retriever())


agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory,

)

# %%
agent(query)
agent("how can chain of thought be applied to analytics?")
agent("how can chain of thought be applied to entetertainment")
# %%
# from pprint import pprint

# pprint(conversational_memory.entity_store.store)


# %%


Error in on_chain_start callback: 'name'


{
    "action": "Knowledge Base",
    "action_input": "Chain of thought is a term used to describe the way in which our minds move from one idea to another. It refers to the way in which our thoughts are linked together, and how one thought can lead to another. The chain of thought can be influenced by a variety of factors, including our emotions, memories, and experiences."
}
Observation: What is the chain of thought?
Thought:

Error in on_chain_start callback: 'name'


{
    "action": "Final Answer",
    "action_input": "The chain of thought refers to the way in which our minds move from one idea to another. It describes the way in which our thoughts are linked together, and how one thought can lead to another. The chain of thought can be influenced by a variety of factors, including our emotions, memories, and experiences."
}

> Finished chain.
{
    "action": "Knowledge Base",
    "action_input": "How can the chain of thought be applied to analytics?"
}
Observation: I'm sorry, but I cannot find any information in the given context about how the chain of thought can be applied to analytics. The context only mentions the use of language models and few-shot exemplars for planning tasks of a SayCan robot. If you have any other questions, I'll do my best to help!
Thought:

Error in on_chain_start callback: 'name'


{
    "action": "Final Answer",
    "action_input": "I apologize for the confusion earlier. To answer your question, the chain of thought can be applied to analytics by using it to understand how people think and make decisions. By analyzing the chain of thought, analysts can gain insights into how people approach problems and make decisions, which can be used to improve decision-making processes and develop more effective strategies. Additionally, the chain of thought can be used to identify patterns and trends in data, which can be used to make predictions and inform business decisions. Overall, the chain of thought is a valuable tool for understanding human behavior and decision-making, which can be applied to a wide range of analytical tasks."
}

> Finished chain.
{
    "action": "Knowledge Base",
    "action_input": "How can the chain of thought be applied to entertainment?"
}
Observation: The chain of thought presented in the context suggests that entertainment-based perceptions 

{'input': 'how can chain of thought be applied to entetertainment',
 'chat_history': [HumanMessage(lc_kwargs={'content': 'how does chain of thought work?'}, content='how does chain of thought work?', additional_kwargs={}, example=False),
  AIMessage(lc_kwargs={'content': 'The chain of thought refers to the way in which our minds move from one idea to another. It describes the way in which our thoughts are linked together, and how one thought can lead to another. The chain of thought can be influenced by a variety of factors, including our emotions, memories, and experiences.'}, content='The chain of thought refers to the way in which our minds move from one idea to another. It describes the way in which our thoughts are linked together, and how one thought can lead to another. The chain of thought can be influenced by a variety of factors, including our emotions, memories, and experiences.', additional_kwargs={}, example=False),
  HumanMessage(lc_kwargs={'content': 'how can chain of th

In [259]:
from langchain.agents.agent_toolkits import (
    create_vectorstore_agent,
    VectorStoreToolkit,
    VectorStoreInfo,
)
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
vectorstore = Pinecone(index, embeddings.embed_query, "text")

vectorstore_info = VectorStoreInfo(
    name="knowlege_base",
    description="latest concepts, research and applications",
    vectorstore=vectorstore
)
toolkit = VectorStoreToolkit(vectorstore_info=vectorstore_info, llm=llm)
agent_executor = create_vectorstore_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

In [260]:
agent_executor(query)

Error in on_chain_start callback: 'name'


 I need to find out what tree-of-thoughts is
Action: knowlege_base
Action Input: what is tree-of-thoughts?

TypeError: replace() argument 2 must be str, not Index

In [241]:
agent_executor.run("explain entertainment science")

Error in on_chain_start callback: 'name'


 I should use the knowlege_base tool
Action: knowlege_base
Action Input: explain entertainment science
Observation:  Entertainment science is the study of how people interact with entertainment media, such as movies, television, video games, and music. It looks at how people use these media to create meaning, how they respond to them emotionally, and how they use them to shape their identities. It also looks at how entertainment media can be used to influence behavior and attitudes.
Thought: I now know the final answer
Final Answer: Entertainment science is the study of how people interact with entertainment media, such as movies, television, video games, and music. It looks at how people use these media to create meaning, how they respond to them emotionally, and how they use them to shape their identities. It also looks at how entertainment media can be used to influence behavior and attitudes.

> Finished chain.


'Entertainment science is the study of how people interact with entertainment media, such as movies, television, video games, and music. It looks at how people use these media to create meaning, how they respond to them emotionally, and how they use them to shape their identities. It also looks at how entertainment media can be used to influence behavior and attitudes.'

In [2]:
import os
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

openai = OpenAI(
    model_name="text-davinci-003",
    openai_api_key=OPENAI_API_KEY
)

llm = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

pinecone.init(api_key=os.getenv("PINECONE_API_KEY"), environment=os.getenv("PINECONE_ENV"))
index_name = "langchain-demo"
index = pinecone.Index(index_name)

from langchain.agents.agent_toolkits import (
    create_vectorstore_agent,
    VectorStoreToolkit,
    VectorStoreInfo,
)
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

/Users/savageabscbn/.pyenv/versions/3.11.3/lib/python3.11/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [14]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain

def get_response_from_query(ai_query, db_query, namespace, k=10):
    """
    gpt-3.5-turbo can handle up to 4097 tokens. Setting the chunksize to 1000 and k to 4 maximizes
    the number of tokens to analyze.
    """

    db = Pinecone.from_existing_index(index_name, embeddings, namespace=namespace)
    docs = db.similarity_search(db_query,k=k)
    docs_page_content = " ".join([d.page_content for d in docs])
    print(docs_page_content)
    chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.2)

    # Template to use for the system message prompt
    template = """
        Act as an expert in {topic}. You answer questions and provide insight into the content.
        # content
        {docs}
        # constraints
        Only use the factual information from the content to form a response.
        If you feel like you don't have enough information to respond truthfully, say "I don't know".
        Your responses should be verbose and detailed.
        """

    system_message_prompt = SystemMessagePromptTemplate.from_template(template)

    # Human question prompt
    human_template = "{question}"
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

    chat_prompt = ChatPromptTemplate.from_messages(
        [system_message_prompt, human_message_prompt]
    )

    chain = LLMChain(llm=chat, prompt=chat_prompt)

    response = chain.run(question=ai_query, docs=docs_page_content, topic=namespace)
    response = response.replace("\n", "")
    return response, docs


db_query = "chain of thought experiment prompt"
ai_query = "Summarise key points"
namespace='ai-research'
response, docs = get_response_from_query(ai_query,db_query,namespace)
print(docs[0].metadata)
print(response)


# chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0,openai_api_key=OPENAI_API_KEY)
# chain = load_qa_chain(llm, chain_type="stuff", verbose=True)

# query = "what is tree of thought?"
# docs = vectorstore.similarity_search(query, k=4, namespace='ai-research')
# docs_page_content = " ".join([d.page_content for d in docs])
# print(docs_page_content)
# chain.run(input_documents=docs_page_content, question=query)

Maxwell Nye, Anders Johan Andreassen, Guy Gur-Ari, Henryk Michalewski, Jacob Austin, David Bieber, David Dohan, Aitor Lewkowycz, Maarten Bosma, David Luan, et al. 2021. Show your work: Scratchpads for intermediate computation with language models. arXiv preprint arXiv:2112.00114.

Long Ouyang, Jeff Wu, Xu Jiang, Diogo Almeida, Carroll L. Wainwright, Pamela Mishkin, Chong Zhang, Sandhini Agarwal, Katarina Slama, Alex Ray, et al. 2022. Training language models to follow instructions with human feedback. arXiv preprint arXiv:2203.02155.

Arkil Patel, Satwik Bhattamishra, and Navin Goyal. 2021. Are NLP models really able to solve

simple math word problems? NAACL.

Matthew E. Peters, Mark Neumann, Mohit Iyyer, Matt Gardner, Christopher Clark, Kenton Lee, and

Luke Zettlemoyer. 2018. Deep contextualized word representations. NAACL.

Xinyu Pi, Qian Liu, Bei Chen, Morteza Ziyadi, Zeqi Lin, Yan Gao, Qiang Fu, Jian-Guang Lou, and

Weizhu Chen. 2022. Reasoning like program executors. arXiv prepr

In [271]:
db = Pinecone.from_existing_index(index_name, embeddings, namespace='')
query="what is chain of thought"
docs = db.similarity_search(query,k=4)
docs_page_content = " ".join([d.page_content for d in docs])
print([d.metadata for d in docs])
docs_page_content

[{'source': './2019_Book_EntertainmentScience.pdf'}, {'source': './2019_Book_EntertainmentScience.pdf'}, {'source': './2019_Book_EntertainmentScience.pdf'}, {'source': './2019_Book_EntertainmentScience.pdf'}]


'(cid:84) (cid:18)\n\n(cid:3)\n\n(cid:84)\n\n(cid:17) (cid:3)\n\n(cid:35)(cid:79)(cid:78)(cid:83)(cid:85)(cid:77)(cid:69)(cid:82)(cid:3)\n\n(cid:35)(cid:79)(cid:78)(cid:83)(cid:85)(cid:77)(cid:69)(cid:82)(cid:3)\n\n(cid:35)(cid:79)(cid:78)(cid:83)(cid:85)(cid:77)(cid:69)(cid:82)(cid:3)\n\n(cid:38)(cid:73)(cid:88)(cid:69)(cid:68)(cid:3)(cid:67)(cid:79)(cid:77)(cid:80)(cid:69)(cid:78)(cid:83)(cid:65)(cid:84)(cid:73)(cid:79)(cid:78)(cid:3)\n\n(cid:50)(cid:69)(cid:86)(cid:69)(cid:78)(cid:85)(cid:69)(cid:3)(cid:83)(cid:72)(cid:65)(cid:82)(cid:73)(cid:78)(cid:71)(cid:3)\n\nE\n\nS\n\nFig. 5.6 Consumer-related revenue allocation models for entertainment products Notes: Authors’ own illustration. The white-colored arrows represent the first transaction/payment for an entertainment product (“t1”), the yellow-colored arrow is the subsequent transaction/payment (“t2”).\n\nDirect Distribution of Entertainment Generating Revenues from Consumers\n\nOur value creation framework highlights the critical

In [55]:
pinecone.delete_index(index_name)

In [6]:
# %%
import os
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from pydantic import BaseModel, Field, validator
from dotenv import load_dotenv
from typing import Optional, Type
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun
from typing import Literal
from common_util.namespaceEnum import PineconeNamespaceEnum
from enum import Enum
from langchain import LLMChain, PromptTemplate, SerpAPIWrapper
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain

In [7]:
load_dotenv()
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

openai = OpenAI(
    model_name="text-davinci-003",
    openai_api_key=OPENAI_API_KEY
)

llm = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

pinecone.init(api_key=os.getenv("PINECONE_API_KEY"), environment=os.getenv("PINECONE_ENV"))
index_name = "langchain-demo"

In [35]:

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
index = pinecone.Index(index_name)
vectorstore = Pinecone(index, embeddings.embed_query, "text")
db = Pinecone.from_existing_index(index_name, embeddings, namespace='ai-research')

# docs = db.similarity_search(db_query,k=5)
index.describe_index_stats()


{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ai-research': {'vector_count': 295}},
 'total_vector_count': 295}

In [50]:
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
llm = OpenAI(temperature=0)

question_generator = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT, verbose=True)
doc_chain = load_qa_chain(llm, chain_type="map_reduce", verbose=True)

chain = ConversationalRetrievalChain(
    retriever=db.as_retriever(),
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    verbose=True
)

In [43]:
retriever = db.as_retriever()
docs = retriever.get_relevant_documents("what is chain of thought")
# docs = db.similarity_search("what is chain of thought",k=5)
print(docs[3])

lc_kwargs={'page_content': '24\n\nTable E13: Basis questions of the Coin Dataset\n\nQ: A coin is heads up. Miguel Angel flips the coin. Saul does not flip the coin. Brady does not flip the coin. Darryl flips the coin. Is the coin still heads up? Note that "flip" here means "reverse". A: Let’s think step by step. Miguel flips the coin, so it’s now tails up. Saul doesn’t flip the coin, so it remains tails up. Brady doesn’t flip the coin, so it remains tails up. Darryl flips the coin, so it’s now heads up again. Q: A coin is heads up. Jacky flips the coin. Socorro flips the coin. Mark flips the coin. Wanda does not flip the coin. Is the coin still heads up? Note that "flip" here means "reverse". A: Let’s think step by step. Jacky flips the coin, so it’s now tails up. Socorro flips the coin, so it’s now heads up again. Mark flips the coin, so it’s now tails up again. Wanda does not flip the coin, so it remains tails up. Q: A coin is heads up. Claudia flips the coin. Cole does not flip the 

In [44]:
chat_history = []
query = "Explain the chain of thought application to AI"
result = chain({"question": query, "chat_history": chat_history})

Error in on_chain_start callback: 'name'



> Finished chain.


In [52]:
chat_history = [(query, result["answer"])]
query = "What is the tree of thought prompting process?"
result = chain({"question": query, "chat_history": chat_history})

Error in on_chain_start callback: 'name'
Error in on_chain_start callback: 'name'


Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: How does tree of thought work with LLMs?
Assistant:  I don't know.
Follow Up Input: What is the tree of thought prompting process?
Standalone question:

> Finished chain.


Error in on_chain_start callback: 'name'
Error in on_chain_start callback: 'name'


Prompt after formatting:
Use the following portion of a long document to see if any of the text is relevant to answer the question. 
Return any relevant text verbatim.
[29] H. Touvron, T. Lavril, G. Izacard, X. Martinet, M.-A. Lachaux, T. Lacroix, B. Rozi`ere, N. Goyal, E. Hambro, F. Azhar, et al. Llama: Open and efﬁcient foundation language models. arXiv preprint arXiv:2302.13971, 2023.

[30] S. Verma, J. Fu, S. Yang, and S. Levine. Chai: A chatbot ai for task-oriented dialogue with ofﬂine reinforcement learning. In Proceedings of the 2022 Conference of the North American Chapter of the Association for Computational Linguistics: Human Language Technologies, pages 4471–4491, 2022.

[31] E. Wallace, N. Tomlin, A. Xu, K. Yang, E. Pathak, M. Ginsberg, and D. Klein. Automated

crossword solving. arXiv preprint arXiv:2205.09665, 2022.

[32] L. Wang, W. Xu, Y. Lan, Z. Hu, Y. Lan, R. K.-W. Lee, and E.-P. Lim. Plan-and-solve prompting:

Improving zero-shot chain-of-thought reasoning by large l

Error in on_chain_start callback: 'name'
Error in on_chain_start callback: 'name'



> Finished chain.
Prompt after formatting:
Given the following extracted parts of a long document and a question, create a final answer. 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.

QUESTION: Which state/country's law governs the interpretation of the contract?
Content: This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an  injunction or other relief to protect its Intellectual Property Rights.

Content: No Waiver. Failure or delay in exercising any right or remedy under this Agreement shall not constitute a waiver of such (or any other)  right or remedy.

11.7 Severability. The invalidity, illegality or unenforceability of any term (or part of a term) of this Agreement shall not affect the continuation  in force of the remainder of the 

In [54]:
print(chat_history)

[('How does tree of thought work with LLMs?', " I don't know.")]


In [3]:
from langchain.document_loaders import YoutubeLoader
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=PAVeYUgknMw")
transcript = loader.load()

In [4]:
print(transcript)

[Document(lc_kwargs={'page_content': "amid the dozens of papers that have come out in the last 10 days there were a couple that butt the trend they showcased how models as powerful as gpt4 could fail at some fairly basic tasks I then set about doing hundreds of my own experiments and have found examples I would say even whole categories of my own that are pretty Illuminating my channel is dedicated to covering the exponential growth in the power of these models but we can still learn a thing or two from their surprising failure modes let's start with some of the simplest examples and end with the very best question write a sentence with the final word fear to repeat the last word in the answer sentence must be in quotes fear answer the only thing we have to fear is fear itself now I don't know about you but I don't think the last word in that sentence is fear this example was inspired by the memo trap which was found in the inverse scaling paper that I'm going to talk more about and it

In [27]:
with open('amazontranscript.txt', 'r') as file:
    file_text = file.read()

In [2]:
from langchain import PromptTemplate

template = """
Act as expert team manager named Alex. Your task is to critically Assess and score the performance of the team member named Noel for each Assessment Criteria.
# Constraints
You will be provided with Noel's Checkpoint Transcript. You assessment is based on the provided Transcript. Mark an Assessment Criteria "N/A" if it has no supporting content in the Transcript.

# Assessment Criteria
## 3-Month Plan

Provide a project plan in 2 weeks showing 3-month CDP project implementation with milestones and timeline.
Sprint Work Plan

Regular cadence for sprint review and work plan update, reflecting previous sprint and its impact on project milestones.
## CDP Data Schema

Design a schema fitting CDP needs including necessary user, customer, custom details, and potential extra properties.
## Execution

Facilitate collaboration for data engineering practices, aligning with data team for CDP value promotion. Maintain excellence, adjust plans for change, and transform objectives into manageable strategies and tasks. Align resources for effective result delivery.
## Excellence - Judgment & Results

Exercise judgment for best returns balancing speed, quality, and cost. Prioritize long-term organization interests over short-term results. Make informed decisions using available data. Derive fulfillment from achievements and problem-solving.
## Excellence - Innovation & Change

Explore innovative ways for continuous improvement. Champion breakthrough ideas. Develop unconventional, untested solutions.
## Teamwork - Ownership

Take accountability for role, output, and decisions.
## Leadership

Inspire and guide others towards goals. Set and communicate MBOs and standards. Encourage risk-taking environment.
## Communication

Communicate clearly, adjust language for audience. Listen and relate well to others across all levels and backgrounds.
## Intellectual Capacity & Analytical Skills

Demonstrate knowledge acquisition ability, logical reasoning, and abstract concept understanding. Identify root causes and generate original thoughts.
## Strategic Thinking & Business Acumen

Understand short and long-term business impacts of decisions. Maintain a long-term, big picture view. Develop appropriate business models considering SWOT.

# Checkpoint Transcript
{query}

# Assessment
"""

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

In [3]:
def chunk_text(text, chunk_size=4000, overlap=200):
    lines = text.split('\n')
    chunks = []
    current_chunk = []
    word_count = 0
    for line in lines:
        line_words = line.split()
        if word_count + len(line_words) <= chunk_size:
            current_chunk.append(line)
            word_count += len(line_words)
        else:
            chunks.append('\n'.join(current_chunk))
            overlap_lines = current_chunk[-overlap:]
            current_chunk = overlap_lines + [line]
            word_count = sum(len(l.split()) for l in current_chunk)
    if current_chunk:
        chunks.append('\n'.join(current_chunk))
    return chunks

In [4]:
full_text="""

Noel: There was a.
 Request to have the other enterprise, the enterprise data sources assess.
 Or the uh, CDP, and also be integrated into the.
 Uh architectural diagram.
 So I did an assessment of the data sources for enterprise and I use uh, this one, the architectural blueprint that was done by Luke Adikanda.
 So I look at this initially for the sources for enterprise and then I also had consultations with Mom Gladys Belmonte.
 So from what I have a research and from uh inputs also from I'm glad this we uh found out that for the data sources for enterprise like all of the listed so for faithfully integrated traffic system RMS resource management Services DBS booking and distribution system IPS integrated payroll system the terms time keeping a sentence each race human resource information System CIDG which is more on revenue reporting non direct digital ads I think this is uh also not related to staff then the OMS order management system and the SAP financials all of them cannot be included in the CDP since they uh are not holding any customer related or.
 Subscriber related data like for example for OMS and for kids.
 Uh, the old data, but mostly for our advertisers and clients.
 So for the others here in the enterprise, it would only hold employee data, which I think, uh it's a very low in count and what we are targeting for the CDP is more of our uh customers.
 And then when I asked Mom Gladys, with regards to the.
 Uh.
 Customer related data is it's uh.
 If there's really a no customer related data or subscription data, she just mentioned that before.
 I think before our franchise got revoked, we had a set or a list of costumer data which we can be obtained from the following.
 But from the following, uh, sources have already been decommissioned.
 Like for example a TV plus Kidzania Skype table, Intel EDS store and they were part of they need something like a single customer view where in that single customer view they had the implementation of the global deduplication process.
 So they are not available now and for some existing LOB's like RP paper view, IPTV, TVOTVO promos at vivia they are existing but the data cannot be utilized since the data it's probably a 2020 and earlier.
 So since 2020, they have not been updating the data, so they are now subject for request for.
 Uhm decommissioning.
 So uh, that it's all for the assessment of the enterprise.
 Data sources.
 So with that assessment, uh, we will just uh push through with the.
 Uh CDB, a phase one which is?
 Combining uh data coming from amplitude, MTV and E Virgin.
 So for that one.
 This is what has been accomplished for the MTV to Mongo DB integration.
 So we have a I have created.
 We have upgraded the extra and that mom.
Jane: Sorry, just to clarify.
 So in the phase one that you're saying is would be the sources would be coming from amplitude MTV and you version.
Noel: Alright, evergent.
Jane: Right.
Noel: Yes ma'am.
Jane: I I don't think that was, uh, enumerated in the plan before, right.
 I remember just seeing amplitude.
Noel: Uh, it was enumerated, ma'am, for the face.
Jane: Is it in the architecture that we were we were reviewing?
Noel: Uh, yes, ma'am.
 But for the enterprise, that was the one that uh you requested.
 If I could already.
 Include.
Jane: Sorry, I did not specifically say go look at enterprise.
 I just said.
 Is that the entire CDP?
 Because you're referring to it as phase one.
Noel: Yes, ma'am.
 If you could recall, you said that, uh, for the architecture it should incorporate he hold picture.
 So that's why I also took a look at the current enterprise data landscape.
 As mentioned, based on the assessment and based from consultations with.
 Mum.
 Dad sent her team for now.
Jane: Umm.
Noel: It's not feasible to include any enterprise data so so we will stick with.
 Uh.
 Amplitude, MTV and EVERGENT or uh the CDP?
Jane: Are you good with that, Alex?
Alexander: Yeah, I think it's, uh, a good idea to start with the high level.
 So you when you're designing the the architecture.
 You have got some consideration for the future.
 Umm.
 And for the for the deliverables for phase one then that's that's amplitude and I'm TV and a virgin, yes.
 Some of these other items might come through in a in another phase.
Jane: Yeah.
 My question was to clarify the phase one.
 So by putting phase one, meaning there would be several phases, right?
Noel: Uh.
 So so for OK.
Jane: So to your point, if that's, yeah, what we wanna see is an overview, overall design, then you, what else is not including face one.
Noel: So for now, ma'am.
Jane: So that's the design that we're looking for, which?
Noel: Uh, yes, ma'am, so.
 Basically what I I had in mind was that the phase one would be the.
 Deliverables, including the amplitude, MTV NE vergent with the following cases reserved for the enterprise data.
 But since the enterprise data is not is no longer feasible, So what most likely would happen is that, uh, phase one is the uh deliverable for initial deliverable for amplitude evergent and MTB, with a possible phase two for optimization based on how the.
 CDP behaves in production.
Jane: I think more than the enterprise because that doesn't have any, umm consumer.
 It's maybe IPTV this that we need to consider.
Alexander: Umm.
Jane: After phase one.
Noel: We also checked mum with we IPTV.
 I'll double check on that one but from uh A.
 My consultation with the yeah.
 I'm.
 I'm glad it's she mentioned that IPTV also has a obsolete data, so one of the considered old LOB, but I'll also double check on that one and she mentioned that there's also a possibility.
 I'll also look into.
 I don't know if you're familiar with you mentioned star Hun.
 We're in like, for example, candidates applying for reality TV shows like Pinoy Big Brother would.
 And.
Jane: Maybe let's start with what's your definition of customer or what is the CDP for, right?
Noel: I.
Jane: We're mixing everything there because it's CDP stands for customer.
 Then let's let's make sure that we're aligned with that.
Noel: Yes, ma'am.
 So our.
 Yes. No.
Jane: Now if customer is specific to digital consumers, then my thoughts would be I want TFC consumers KTX or I want tickets.
 Umm, whatever you call it now and then IPTV.
Noel: Yes, ma'am.
 Yes, ma'am.
Jane: Right.
Noel: So it's mostly with the our digital consumers.
 You just have to answer my subscribers.
Jane: So why are we looking into StarHub?
Noel: Uh, that's just one of the suggestions of a month.
Jane: And why are we disregarding ITV?
Noel: Or IPTV, as mentioned, I'll double check on that one, although Mom Gladys mentioned that the data is.
Jane: Umm.
Noel: Obsolete for IPTV have being one of the old a hello bit although I double check.
 Ohh or probably OK.
Jane: I think you're mixing it with the TV plus or with IPTV is not local.
 It's or.
 It's a running business.
Noel: OK, ma'am, I'll double check on that one.
 Maybe I just got the.
 The notes are mixed up.
 I thank you for that one.
Jane: In fact, there's said directly relationship between IPTV and I want TFC.
Noel: OK, ma'am.
 So if that that baggage, then definitely it would be included in the phase two implementation.
 So for the uh integrations of uh, MTV AMPLITUDE and evergent development has already started for the three and MTV has been, uh finished as far as the initial.
 Development is concerned so.
 We have uh, I have created the functions for extract and transform a function.
 So for this one, this needs to be optimized since the API key needs to be.
 Uh masco.
 I'll make a masking of function for this one so that the API P is, uh, not exposed, so just for purposes of showing what has been I mean so uh this function that extracts data from megaphone TV and this will be getting the important attributes such as a user ID, device ID, username and user email.
 Although as mentioned by Alex.
 Uh, when I checked with regards to birthday and gender, they are not included in the MTV.
 So for.
 Uh.
 Personal data.
 Only username and the email that can be included.
 So there's a transformation involved and the output for this one.
 It's a like this one.
 So you could see here user ID, device ID, name and email and once the extraction and the transformation process has been, uh, completed, we now move to the loading of the data to the mongo.
 The so for now it's run in the local holes, although by next week I'll be doing the test with more test data in the dev environment.
 So as you could see, based on the output data saved to Mongo DB successfully and then here is the.
 Uh, I'll put when done in uh, when steam in the.
 Mongo DB local database so that is for Mongo DB and the target for the next print would be the completion of the initial uh extraction transformation and loading for amplitude and evergent while also enhancing uh.
 This one for uh for empty.
Jane: So with all that said, going back to your proposed plan, what have you accomplished?
 The way I see it, there's nothing there.
Noel: From this one Mama, it can be seen that the template or the initial prototype for the.
 Loading of the but data from the data source from MTV to Mongo DB has been, uh proven by this.
 Uh.
 Functions in the outputs that I am shown.
 So for the next few weeks, the testing with the more data, the actual data and also with the completion of the amplitude and the.
 MTV integrations, once loaded into Mongo DB, the next space, the next part would be consolidation of of those data.
 So in this sense, there has been an accomplishment where in uh it has been shown that, uh, data coming from megaphone can impact be extracted, transformed and loaded to a mongo DB database for further use in the final integration procedure, including unfiltered and uh evergent.
Alexander: Would you consider this under the line?
 Uh, and if you look at the PIP deliverables, we've got.
 Uh amplitude data uploaded to CDP, and under that there's Mongo DB service.
Noel: Uh, yeah.
Alexander: Add the line insert user properties so you've got insert user properties completed locally for a sample of MTV data.
Noel: Ah yes, Alex.
Alexander: This is also this is a a function, not a service, so that's that's the Mongo DB service.
Noel: OK, I'll note that done that well.
Alexander: Is that?
 At the moment that that function is run.
 Manually when you have a.
 When you pause it in a file, is that?
 Is that how it's working?
Noel: Yes, for now.
 It's uh, manually, so the next step would be to.
 Uh, do it in an automated function.
 That passion, similar to what was done before with the implementations of the S3, Ethel and the using Lambda.
 So it's going to have that and kind of uh, next step procedure.
Alexander: OK.
Noel: Yes.
 So also kindly take note that or the uh for this phase included in recovery plan now, uh, I gave a timeline that there would be an extension up to.
 Ah, the the latter part of September, due to what happened with the uh datazoom not being uh feasible to do the Mongo DB connector.
 So with the current uh development, I see that the we are still on track for that.
 Uh profile.
 Delivery by sometime at the end of September.
Alexander: OK.
 Continue, Noel.
Noel: OK, so uh, those are all the.
 Uh, important update.
 So I'll just show the targeted.
 Uh milestones for this.
 Updating speeds are just unshare and open a new document.
 Can you see my screen now?
Jane: Yeah.
Noel: OK, so for the recovery plan, so this have these are the things that have been accomplished.
 So for the Sprint one, what was completed was the assessment of the enterprise data sources for Sprint to be completion or be successful extraction in the transformation of MTV data sources and be successful loading of MTB to MongoDB.
 Although, let's discussed that this is still for a few test data in this still done manually, so this will be, uh, updated in the print three.
 So first print three uh targeting to uh, complete the following.
 So the BI form accomplishment can be free ARB solution design approval.
 Then the ARB solution design approval.
 Then by that time, there's already a successful loading of amplitude test data come up with the best for loading of evergent test data for Mongo DB and completion of the API masking function in MTV as well as.
 He automation of the.
 Uh ETL process?
Jane: So how do these ohm accomplishments?
 Umm.
 Relate to the overall schedule and milestones.
Noel: Up so for the overall schedule.
 Uh, this is the uh uh over all the.
 Overall schedule.
 So I just by the way for this schedule I made an extra, uh, one week just to be able to include the additional week that was used for the assessment of the enterprise.
 Uh data sources.
 So before it was scheduled to end on September.
 Plenty 2 back to be done for the enterprise data sources to have a whole picture.
 Architecture.
 So I just added a another week but as discussed the previously targeting the complete everything uh on or before the end of September.
Jane: So question on the green colored ohh blind snow is this from your perspective?
Noel: I'm.
Jane: Who is certifying that these deliverables have been accomplished?
Noel: I'm that.
 That's from my perspective.
Jane: But who should be certifying that these are indeed completed?
Noel: Up for that one.
 How far the others for the other items like for the three ARB?
Jane: Or what's your evidence that it's completed?
 Yeah.
Noel: Uh, with regards to like what I just presented with the assessment of a enterprise data source, so I'll just consolidate, ma'am via list of evidences on that one.
 Helping them.
Jane: So on a party line you link the outcome so we can also review that Alex.
Noel: I don't know.
Alexander: Yeah.
Noel: Umm.
Jane: Then we cannot knowledge, it's it's done depend and we can also rate it accordingly, whether it's it's not just essentially completed right, we wanna make sure that the the quality of the output is also sufficient.
 And then.
Noel: I'll make my noted on that.
Jane: Well, we talked about an extension, Noel, the way this works is you have to formally appeal for that extension and we will now take a look at whether that uh extension will be necessary.
Noel: OK, ma'am.
Jane: No.
 No, no man.
Noel: So for that one, is there a a standard document or template for that time or just in OK ma'am?
Jane: Just refer it to the PIP, the original plan, the revised why do you why you requesting for an extension?
 Because essentially you have to justify it.
Noel: Yes, ma'am.
Jane: Because when we started this, umm, you know, it's also you coming up with the three month plan plan seeing that it's doable and then?
 Midstream you're calling out that it's gonna be delayed.
Noel: OK, ma'am.
Jane: So that has to be put in writing the reasons.
Noel: So for that one for that one, I'll just include you, Alex and Mom *****.
Jane: Yes.
Noel: OK, OK.
Jane: To uh, the request should be course not addressed to me and then just copy for copy for English language.
Noel: Nothing then.
Jane: Alex and Lance, they're like, will also be informed of that.
 They will.
 Uh, take a look at your request there.
 Justification.
 And then we'll talk to you about it.
Noel: Mama.
 Mama, to be specific for the just justification.
 What are the?
 He he points that are you'd like to.
Jane: It's.
 Umm.
Noel: Uh, see, there are just a not narration of the things that happened that caused the delayed.
 Like for example, what happened with the?
 Datazoom proposals is that uh enough.
Jane: Well, it's more of a course, but what should be there would be your how much time you're requesting, right?
Noel: OK.
Jane: So the extension the details of that extension.
 And the reason for umm having those extensions?
Noel: OK.
Jane: Because the baseline is the original three month plan, which you also drafted and then we it it we in that letter it has to be understood clearly why there was a delay which will be the basis of us reconsidering giving you an extension, right.
Noel: So for that one, ma'am, I'll just put in the subject.
 No. Ah.
Jane: Request for extension of.
Noel: Request for extension of PLP.
Jane: Yeah.
Noel: OK, ma'am.
 No, that one, that one.
Alexander: No.
 Have you filled out the PIP assessment?
Noel: Uh, yes.
 Uh, it's mentioned to mangena.
 I focus on the other deliveries, but uh, I will definitely.
 Submitted within the day.
 Uh, apologies for the delay on that.
Alexander: Umm.
Jane: OK.
 And then for this, just make sure that you submit a within today the specific status and then the link to the deliverables for it.
 Now for.
 The milestones that we said that it has been completed so that we have clear reference and they will use that also to you, uh, acknowledge or certify that you know it's been completed and then if there are you know quality concerns then we also visit.
Noel: OK, ma'am.
 And also last question is uh, a link to the the the file sufficient or will or should we have like a SharePoint site or Google?
 Uh, Google Docs.
Jane: Umm.
Noel: OK then.
 OK.
Jane: A link should suffice because that link the manual point to, for example the confluence right and then there will be like.
 It will be dated.
 Right, Alex, if there are changes, we would also see that.
 Is that correct?
Alexander: Yep, yeah, there's a change log.
Jane: There's some change log in there.
Alexander: Special control, yeah.
Noel: OK, I'm not done that.
 I just put all the deliverables and evidence, as in a confluence page, so it's easier to link and find the items.
Alexander: Yeah, a big part of the deliverables is the documentation, Noel, which should be in compliance anyway.
Noel: Uh, yes.
 It's uh in confluence.
 The uh documentation, although I still need to update the some information.
 Uh for the confidence.
 So I'll just include the the list of.
 Yeah.
 Evidence that are screenshots of the evidence that's in the conference.
 Page just applicable.
Jane: Answer submit the request for extension.
 Umm, you know no later than Tuesday.
 So because we also need to discuss and deliberate on that.
Noel: OK.
Jane: And then the self assessment by today.
Noel: Yes, ma'am.
 Yes, ma'am.
 Ma'am, question we did regards to the.
 Uh.
 Request for?
 Extension.
 Is there a possibility that I'm extension may not be, uh, granted.
Jane: Yes.
 It will depend on the reasoning.
Noel: OK mams but for that one who would be the?
 Happy person to during the meeting.
 Besides you, Alex and Magnus, are there other symbols or just the thing that we?
Jane: That's uh, just the three of us.
Noel: OK.
Jane: So we'll look at the your course, your request, you know, why would we consider an extension?
 Will the extension actually achieve the goal?
 So we'll we'll also take a look at where we are, you know in some.
 In terms of your looking into the deliverables that you have completed.
Noel: OK then.
 I and also ma'am, this is not related to the CDP deliverables.
 Mom can be approved the the Mr request that I submitted.
 It's just the duplicate of the original Mr request for the run module that was initially.
 Uh.
 Assigned to you or your direct CBN.
 But this time around I'm the recipient.
Jane: And yeah, I saw your note on that, but and unfortunately I don't, I didn't receive any notification.
 OK, if that can be resent so that can pick it up.
Noel: OK, ma'am.
Jane: So I don't have any idea where to apply.
Noel: OK, man.
Jane: And that's it.
 How would this next week looks like with all these upcoming?
Noel: A.
 For next week, I will definitely be a targeting these two items, PIA form accomplishment and the pre ARB review while at the same time continuing with the.
 Uh developments development for?
 Amplitude and evergent and the improvement of the.
 Uh, MTV, MTV functions, making it automated and incorporating the masking function.
 Although for this one for amplitude and the evergent, this is this most likely would extend up to the end of.
 Uh.
 The following uh.
 Flowing, but definitely for next week, these two are.
 IPC bug.
Jane: OK, so of course I'm looking at an older version of your recovery plan, which states that your item Rule 15.
 No, no, hang on.
 Let's see.
 We are there real 14.
 Is already delayed.
 So we cannot just make adjustment on your own without consulting us because we're relying on what we've also looked at.
 That's why we also want to make sure that things are documented and formalize.
Noel: OK, ma'am, I'll send the a copy of a for this.
 One of the updated club.
Jane: Yeah, last week I say I was asking for a copy of the status reporting Johnson.
 And then yeah, there's nothing else.
 Just make sure because we're going to start, you know, looking into.
 What was the livered so and then the drill down into the output, the quality and all that.
Noel: And then.
Jane: So we'll start the, the, the assess the rating.
Noel: And I'm doing that.
 Also, another topic that's not part of the deliverables, but should have an impact with regard to the.
Jane: Umm.
Noel: Uh and well, physical exam, because if I'm not mistaken.
 It it should it be done at the office or can it just be done with accredited providers of a PE in our area so that I won't be using I I'm scheduled for Thursday next week so I won't be.
Jane: Please check with Mavic.
 I will not be from.
Noel: OK.
Jane: I'm not familiar also with the process, but yeah, go check with my big direction.
Noel: Yes, ma'am, because I'm going to be requesting if it can be done at a clinic near my place.
 So I could have it done on a Saturday or on a Sunday so that I won't consume fine on Thursday.
Jane: Yeah, this talk nothing about you.
Noel: Yes, ma'am.
 OK, done that one.
Jane: OK.
 So send us the your self assessment there.
 A copy of the latest status report with the the link to all the output that we've provide that you finished, and then we'll wait also for your request for extension.
Noel: OK.
 Mama noted on all of those items.
Jane: Anything else? Uh.
 Alex someone.
Alexander: No, I'm good.
Jane: So for next week for the uh, row 13 three, you'll be review.
 Cool.
 Alex, turn you also part of this meeting.
 Where should you be?
Alexander: I have a review.
Jane: Should they be represented?
 Are are you looked in?
Alexander: And not for for the, not for the A, not for AIB.
 Umm, but definitely before.
Jane: This one uh rose 13.
Alexander: Our pre AB review.
 Umm.
Jane: But I was.
 I was thinking it's the session right? Because.
Noel: Offer for this one.
Jane: Are is this gonna be an offline review, Noel, or you setting up a meeting?
Noel: Actually I'm thinking I'm actually thinking of doing it at 2 weeks.
 1st, it's the setting a meeting where all of us are involved, columnar and if Alex is also to be included and all of us, but due to a difference in time zones, if there won't be a common time, I will just be.
 Scheduling a separate sessions and then consolidate the feedback and then once console the feedbacks or consolidated uh, put it into writing via email for concurrence of all parties involved.
Jane: Nothing.
 Just make sure because I don't want you know, the time zone, the availability to become an issue of, well, you getting delayed.
Noel: Yes, ma'am.
Jane: Where we passed that night.
Noel: Yes, ma'am.
Jane: Just make sure I mean Naren, even if he's in a different time zone.
 Can be tapped in a meeting if he needs to be there.
Noel: OK.
Jane: Just make sure that you insist and also be, you know, he's usually available in the morning until our lunchtime.
 Or late evening.
 So the point is, you're the one chasing, so you have to make sure that we can chase them.
Noel: OK, OK.
 OK, no.
Jane: It would be best though, that you sent and an advanced copy so that they they know we can do some and they can spend some time to review before the session.
Noel: Nothing on that one.
Jane: We can you busy in, in, in whatever communication or either meeting.
Noel: OK.
Alexander: Right.
 Thank you, Noel Jane.
Jane: Alright, thank you.
Noel: Thank you, ma'am.
 Thank you, Alex.
 Uh, not happy weekend.
 God bless.
Alexander: You too. Bye.
Jane: OK then.

"""

In [5]:
chunks=chunk_text(full_text)

In [6]:
from common_util.llms import LLM_CHAT_4
from langchain.chains import LLMChain
llm_chain = LLMChain(prompt=prompt_template, llm=LLM_CHAT_4)


In [7]:
action =""
for chunk in chunks:
    action += llm_chain.run(chunk)

In [9]:
template_fin = """
Act as expert team manager named Alex. Your task is to critically assess and score the performance of the team member named Noel. Respond "understood" to confirm you understand the assessment task and the Assessment Criteria.
# Constraints
You will be provided with Noel's Previous Assessments. 
Your Final Assessment is based on the Provided Assessments.
Mark an Assessment Criteria "N/A" if it has no supporting content in the Previous Assessments.

# Criteria
## 3-Month Plan

Provide a project plan in 2 weeks showing 3-month CDP project implementation with milestones and timeline.
Sprint Work Plan

Regular cadence for sprint review and work plan update, reflecting previous sprint and its impact on project milestones.
## CDP Data Schema

Design a schema fitting CDP needs including necessary user, customer, custom details, and potential extra properties.
## Execution

Facilitate collaboration for data engineering practices, aligning with data team for CDP value promotion. Maintain excellence, adjust plans for change, and transform objectives into manageable strategies and tasks. Align resources for effective result delivery.
## Excellence - Judgment & Results

Exercise judgment for best returns balancing speed, quality, and cost. Prioritize long-term organization interests over short-term results. Make informed decisions using available data. Derive fulfillment from achievements and problem-solving.
## Excellence - Innovation & Change

Explore innovative ways for continuous improvement. Champion breakthrough ideas. Develop unconventional, untested solutions.
## Teamwork - Ownership

Take accountability for role, output, and decisions.
## Leadership

Inspire and guide others towards goals. Set and communicate MBOs and standards. Encourage risk-taking environment.
## Communication

Communicate clearly, adjust language for audience. Listen and relate well to others across all levels and backgrounds.
## Intellectual Capacity & Analytical Skills

Demonstrate knowledge acquisition ability, logical reasoning, and abstract concept understanding. Identify root causes and generate original thoughts.
## Strategic Thinking & Business Acumen

Understand short and long-term business impacts of decisions. Maintain a long-term, big picture view. Develop appropriate business models considering SWOT.

# Provided Assessments
{query}

# Final Assessment
"""

prompt_template_fin = PromptTemplate(
    input_variables=["query"],
    template=template
)

In [10]:
llm_chain_fin = LLMChain(prompt=prompt_template_fin, llm=LLM_CHAT_4)


In [11]:
output = llm_chain_fin.run(action)
print(output)

## 3-Month Plan
Score: 2/5

Noel has shown some progress in the project implementation but has not been able to stick to the original 3-month plan. He has requested an extension to the timeline, indicating a lack of planning or unforeseen challenges.

## Sprint Work Plan
Score: N/A

The transcript does not provide enough information to assess Noel's performance on this criterion.

## CDP Data Schema
Score: N/A

The transcript does not provide enough information to assess Noel's performance on this criterion.

## Execution
Score: 3/5

Noel has shown some ability to facilitate collaboration and adjust plans for change. He has made progress in the project implementation, but there are still areas that need improvement, such as adhering to the original timeline.

## Excellence - Judgment & Results
Score: 2.5/5

Noel's judgment and results are mixed. While he has made progress in some areas, he has also requested an extension to the project timeline, which indicates a lack of judgment in pl

In [49]:
from langchain import PromptTemplate

template = """
System {{
You are the Org Transformer. Your expertise lies in transforming business to reach their maximum potential.
You create playbooks that will be followed by your target audience, to level up their organisation.
Your playbook is always informative, direct, and based on evedence of success.
Culture, org structure, and frameworks are some of the many tools at your disposal. You create tools to suit your audience's need.
}}

Audience {{
    Digital media congamorate. 
    Teams include product, data, marketing, sales, revenue, content aquisition, finance.
    Traditionally structured org.
    Hierarchial culture of decision making.
}}

Task {{
    Use the Amazon Presentation Transcript as source material to generate material for your playbook.
    Include relevant details for your target Audience, while remaining concice and accurate to the source.
}}

# Presentation transcript
{query}

# Playbook material
"""

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

In [50]:
llm_chain = LLMChain(prompt=prompt_template, llm=LLM_CHAT_4)


In [31]:
chunks = chunk_text(file_text)
len(chunks)

8

In [51]:
suggestions = ""
for chunk in chunks:
    suggestions += llm_chain.run(chunk)

In [58]:

template_fin = """
System {{
You are the Playbook editor. Your expertise lies in transforming business to reach their maximum potential.
You are given a bunch of playbook Content to summarise edit to deliver to help the reader level up inovation within their organisation.
Finding connections between concepts and their appication comes second nature to you. You put yourself in the shoes of the Audience when editing.
}}

Audience {{
    Digital media congamorate. 
    Teams include product, data, marketing, sales, revenue, content aquisition, finance.
    Traditionally structured org.
    Hierarchial culture of decision making.
}}

Task {{
    Edit the provided playbook Content so that it is ready for Audience consumption.
}}

# Content
{query}

# Playbook final version
"""

prompt_template_fin = PromptTemplate(
    input_variables=["query"],
    template=template_fin
)
llm_chain_fin = LLMChain(prompt=prompt_template_fin, llm=LLM_CHAT_4)


In [56]:
suggestions="""
## Playbook: Transforming Your Digital Media Conglomerate

### 1. Cultivate a Customer-Centric Culture

Emulate Amazon's success by putting the customer at the heart of everything you do. This approach should permeate all teams, from product to marketing, sales, and content acquisition. 

**Action Steps:**
- Instill a customer-first mindset across all teams.
- Regularly collect and analyze customer feedback to understand their needs and preferences.
- Develop products, services, and strategies that cater to these needs and preferences.

### 2. Foster a Culture of Innovation

Promote a culture of innovation akin to Amazon's. Encourage employees to take calculated risks and experiment with new ideas. 

**Action Steps:**
- Encourage teams to experiment with new ideas and approaches.
- Create an environment where calculated risk-taking is rewarded, not penalized.
- Implement a system for tracking and learning from both successful and unsuccessful experiments.

### 3. Implement Effective Decision-Making Frameworks

Adopt a set of guiding principles for decision-making, similar to Amazon's leadership principles. These principles should align with the company's mission and values. 

**Action Steps:**
- Develop a set of principles or values to guide decision-making across your organization.
- Train all employees on these principles or values and how they should be applied in their work.
- Use these principles or values as a guide in performance reviews and other feedback sessions.

### 4. Promote Writing Skills

Value writing skills as Amazon does. Encourage employees to write detailed business plans and essays about their work. This promotes clear thinking and effective communication.

**Action Steps:**
- Encourage employees to enhance their writing skills.
- Implement a system for reviewing and providing feedback on written work.
- Use written documents as a key tool in decision-making and strategy development.

### 5. Implement the Two-Pizza Meeting Rule

Adopt Amazon's "two-pizza rule" for meetings: if a meeting can't be fed with two pizzas, it's too big. This keeps meetings small and productive.

**Action Steps:**
- Limit the size of meetings to ensure they are more focused and productive.
- Encourage teams to communicate and collaborate outside of meetings.
- Review your current meeting practices and look for ways to make them more efficient.

### 6. Distinguish Between One-Way and Two-Way Decisions

Differentiate between one-way decisions (those that are hard to reverse) and two-way decisions (those that can be easily changed). This accelerates decision-making and promotes experimentation.

**Action Steps:**
- Train employees on the difference between one-way and two-way decisions.
- Encourage teams to make two-way decisions quickly and to experiment with new ideas.
- Review one-way decisions carefully to ensure they are the right choice for your organization.

Remember, the goal is to transform your organization into a more agile, innovative, and customer-centric entity. By adopting these strategies, you can help your organization reach its full potential.


Title: A Playbook for Digital Media Transformation: Inspired by Amazon's Leadership Principles

### 1. Prioritize Customer-Centric Innovation:

- Begin with understanding your customer's needs and work backwards to develop innovative solutions. 
- Encourage employees to think creatively and provide them with the necessary resources to test their ideas. 
- Use a structured document like Amazon's PRFQ (Press Release and Frequently Asked Questions) to guide the innovation process.

### 2. Foster a Culture of Experimentation:

- Cultivate an environment where experimentation is encouraged and rewarded. 
- Allow employees to test their ideas, even if the outcomes are uncertain. 
- Manage risk by starting with small, reversible experiments and focus on lowering the cost of failure.

### 3. Implement Agile Organizational Structures:

- Adopt Amazon's "two-pizza teams" concept, keeping teams small and autonomous for quicker decision-making and efficient work. 
- Encourage flexibility across the organization, allowing employees to engage in different areas and add value where needed.

### 4. Leverage Microservices Architecture for Innovation:

- Utilize a microservices architecture to innovate and create new business opportunities quickly. 
- Democratize access to technology and data, enabling more employees to participate in the innovation process.

### 5. Embed Leadership Principles in Culture:

- Implement Amazon's leadership principles as a guide for decision-making at all levels of the organization. 
- Use these principles in hiring, performance reviews, and training to ensure alignment with the company's values and goals.

### 6. Manage Financial Constraints:

- Operate on a principle of frugality, always looking for ways to innovate despite financial constraints. 
- Encourage employees to break their ideas down to the smallest component and validate each step as they move forward.

### 7. Hire and Develop Leaders:

- Implement a rigorous hiring process to ensure that you hire leaders who can drive your culture and principles. 
- Reward employees for innovative thinking and behavior, not just successful outcomes.

By implementing these strategies, your digital media conglomerate can foster a culture of innovation, improve decision-making, and better serve its customers.
"""

In [59]:
print(llm_chain_fin.run(suggestions))

Title: A Blueprint for Digital Media Transformation: Drawing Inspiration from Amazon's Leadership Principles

### 1. Embrace Customer-Centric Innovation:

- Prioritize understanding your customer's needs and reverse-engineer innovative solutions. 
- Foster a culture where employees are encouraged to think creatively and are provided with the necessary resources to test their ideas. 
- Utilize structured documents like Amazon's PRFQ (Press Release and Frequently Asked Questions) to guide the innovation process.

### 2. Cultivate a Culture of Experimentation:

- Establish an environment where experimentation is not just tolerated but celebrated and rewarded. 
- Empower employees to test their ideas, irrespective of the uncertainty of outcomes. 
- Mitigate risk by initiating small, reversible experiments and concentrate on reducing the cost of failure.

### 3. Adopt Agile Organizational Structures:

- Implement Amazon's "two-pizza teams" concept, maintaining teams small and autonomous for